<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-библиотек" data-toc-modified-id="Загрузка-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка библиотек</a></span></li><li><span><a href="#Загрузка-и-изучение-данных" data-toc-modified-id="Загрузка-и-изучение-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка и изучение данных</a></span></li><li><span><a href="#Лемматизация-и-работа-с-регулярными-выражениями" data-toc-modified-id="Лемматизация-и-работа-с-регулярными-выражениями-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация и работа с регулярными выражениями</a></span></li><li><span><a href="#Целевая-переменная-и-признаки" data-toc-modified-id="Целевая-переменная-и-признаки-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Целевая переменная и признаки</a></span></li><li><span><a href="#Разделение-на-обучающую-и-тестовую-выборки" data-toc-modified-id="Разделение-на-обучающую-и-тестовую-выборки-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Разделение на обучающую и тестовую выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-CatBoostClassifier" data-toc-modified-id="Модель-CatBoostClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель CatBoostClassifier</a></span></li><li><span><a href="#Подбор-порога-для-модели-LogisticRegression" data-toc-modified-id="Подбор-порога-для-модели-LogisticRegression-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Подбор порога для модели LogisticRegression</a></span></li></ul></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача: обучить модель классифицировать комментарии на позитивные и негативные на основании набора данных с разметкой о токсичности правок.

Метрика качества *F1* модели должна быть не меньше 0.75.. 

**План проекта**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

### Загрузка и изучение данных

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [3]:
data.sample(10)

,text,toxic
111361,PDchick was NOT my account. QUIT SAYING IT IS!!,0
157981,"February 2008 (UTC)\n\nIt IS wikipedia, the in...",0
23812,or American http://www.voanews.com/content/eur...,0
35947,"Welcome!\n\nHello, حياتنا, and welcome to Wiki...",0
112415,"Question\n\nHi Striver,\n\nIve got a question ...",0
76124,You are a faggot!68.33.41.181,1
61995,Democracy and blue water navy are not interlin...,0
39264,"Can I Eye\nFUCK YOU YOU NEGRO SEE GROW 05:45,...",1
93359,"""\nWell, this is bad... but thanks. I can now ...",0
145346,"""\nYou're the one who's making an """"edit war""""...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [5]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [6]:
print(data[data['toxic']==1].shape)
print(data[data['toxic']==0].shape)

(16186, 2)
(143106, 2)


### Лемматизация и работа с регулярными выражениями 

In [7]:
%%time
# Lemmatize with POS Tag
from nltk.corpus import wordnet
import nltk
nltk.download('averaged_perceptron_tagger')
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
tokens = []
for sentence in data['text']:
    token = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
    tokens.append(' '.join(token))
data['lemm_text'] = tokens



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


CPU times: user 19min 15s, sys: 1min 57s, total: 21min 12s
Wall time: 21min 13s


In [8]:
%%time
def clear_text(text):
    text_new = re.sub(r'[^a-zA-Z]', ' ', text).lower()
    text_split = text_new.split()
    return ' '.join(text_split)

data['lemm_text'] = data['lemm_text'].apply(lambda x: clear_text(x))


CPU times: user 4.1 s, sys: 72 ms, total: 4.17 s
Wall time: 4.18 s


In [9]:
data.sample(5)

,text,toxic,lemm_text
86968,"""\nYou appear to be misunderstanding. Surely ...",0,you appear to be misunderstand surely a sectio...
19270,"""\n\n Ethiopia christian nation? \nSince there...",0,ethiopia christian nation since there be more ...
33106,Removed links \n\nDear Cyberjunkie - can you p...,0,removed link dear cyberjunkie can you please l...
10019,"Nuh-uh, I see track loaders used a lot more on...",0,nuh uh i see track loader use a lot more on ba...
110570,Not really developed enough at all. Might be h...,0,not really developed enough at all might be he...


### Целевая переменная и признаки 

In [10]:
target = data['toxic']
features = data['lemm_text']


### Разделение на обучающую и тестовую выборки

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, stratify=target, random_state=12345)


In [12]:
print(features_train.shape)
print(features_test.shape)

(119469,)
(39823,)


**Вывод:** Данные успешно выгружены и изучены. Наблюдаем сильный дисбаланс классов. Данные по текстам обработаны: проведена лемматизация текста и работа с регулярными выражениями. Определена целевая переменная и признак, данные поделены на обучающую и тестовые выборки.

## Обучение

### Модель LogisticRegression

In [13]:
%%time
pipeline = Pipeline([
           ('vect', TfidfVectorizer(stop_words=stopwords.words('english'))),
           ('clf', LogisticRegression(class_weight='balanced')),
])
parameters = [{
    
    'vect__ngram_range': ((1, 1), (1, 2), (1,3),),
    
}]

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=10, cv=3, scoring="f1")
grid_search.fit(features_train, target_train)
best_score = grid_search.best_score_

print('Оценка F1:', best_score)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3; 1/3] START vect__ngram_range=(1, 1)....................................
[CV 1/3; 1/3] END ..................vect__ngram_range=(1, 1); total time=  36.9s
[CV 2/3; 1/3] START vect__ngram_range=(1, 1)....................................
[CV 2/3; 1/3] END ..................vect__ngram_range=(1, 1); total time=  29.4s
[CV 3/3; 1/3] START vect__ngram_range=(1, 1)....................................


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3; 1/3] END ..................vect__ngram_range=(1, 1); total time=  42.7s
[CV 1/3; 2/3] START vect__ngram_range=(1, 2)....................................
[CV 1/3; 2/3] END ..................vect__ngram_range=(1, 2); total time= 1.6min
[CV 2/3; 2/3] START vect__ngram_range=(1, 2)....................................
[CV 2/3; 2/3] END ..................vect__ngram_range=(1, 2); total time= 1.5min
[CV 3/3; 2/3] START vect__ngram_range=(1, 2)....................................


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3; 2/3] END ..................vect__ngram_range=(1, 2); total time= 2.0min
[CV 1/3; 3/3] START vect__ngram_range=(1, 3)....................................
[CV 1/3; 3/3] END ..................vect__ngram_range=(1, 3); total time= 2.4min
[CV 2/3; 3/3] START vect__ngram_range=(1, 3)....................................
[CV 2/3; 3/3] END ..................vect__ngram_range=(1, 3); total time= 2.9min
[CV 3/3; 3/3] START vect__ngram_range=(1, 3)....................................
[CV 3/3; 3/3] END ..................vect__ngram_range=(1, 3); total time= 1.9min
Оценка F1: 0.7398584988927892
CPU times: user 8min 34s, sys: 6min 16s, total: 14min 51s
Wall time: 14min 52s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting 3 folds for each of 3 candidates, totalling 9 fits

Оценка F1: 0.7398584988927892

### Модель CatBoostClassifier

In [ ]:
pipeline = Pipeline([
           ('vect', TfidfVectorizer(stop_words=stopwords.words('english'))),
           ('cat', CatBoostClassifier(n_estimators=50)),
])
parameters = [{
    
    'vect__ngram_range': ((1, 1), (1, 2), (1,3),),
            
}]

grid_search_cat = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=10, cv=3, scoring="f1")
grid_search_cat.fit(features_train, target_train)
best_score = grid_search.best_score_

print('Оценка F1:', best_score)

Оценка F1: 0.7137507298178837

### Подбор порога для модели LogisticRegression

In [14]:
def to_labels(pos_probs, threshold):
 return (pos_probs >= threshold).astype('int')

In [15]:
%%time
# predict probabilities
yhat = grid_search.best_estimator_.predict_proba(features_train)
# keep probabilities for the positive outcome only
probs = yhat[:, 1]
# define thresholds
thresholds = np.arange(0, 1, 0.001)
# evaluate each threshold
scores = [f1_score(target_train, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = np.argmax(scores)
print('Threshold=%.3f, F1-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.620, F1-Score=0.85766
CPU times: user 34.6 s, sys: 207 ms, total: 34.8 s
Wall time: 34.8 s


**Вывод:** Обучены несколько моделей с различными гиперпараметрами: LogisticRegression, CatBoostClassifier. Метрика качества выбрана F1. 

Наилучший результат показала модель LogisticRegression c подбором порога: Threshold=0.620, F1-Score=0.85766.

## Тестирование лучшей модели

In [16]:
%%time
# predict probabilities
yhat = grid_search.best_estimator_.predict_proba(features_test)
# keep probabilities for the positive outcome only
probs = yhat[:, 1]
# define thresholds
thresholds = np.arange(0, 1, 0.001)
# evaluate each threshold
scores = [f1_score(target_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = np.argmax(scores)
print('На тестовой выборке:', 'Threshold=%.3f, F1-Score=%.5f' % (thresholds[ix], scores[ix]))

На тестовой выборке: Threshold=0.675, F1-Score=0.78215
CPU times: user 11.7 s, sys: 85.1 ms, total: 11.8 s
Wall time: 11.8 s


**Вывод:** Проведено тестирование модели LogisticRegression на тестовых данных: лучшая метрика F1-Score=0.78215 для порога, равного 0.675.

## Проверка модели на адекватность

In [20]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), ngram_range=(1,2), max_features=30000)
features_train_vectorized = vectorizer.fit_transform(features_train)
dummy_clf = DummyClassifier(strategy='constant', constant=1)
dummy_clf.fit(features_train_vectorized, target_train)
dummy_predictions = dummy_clf.predict(features_train_vectorized)
f1 = f1_score(target_train, dummy_predictions)
print("F1 для фиктивного классификатора:", f1)

F1 для фиктивного классификатора: 0.1844858634287929


## Выводы

Проведено исследование набора данных с разметкой о токсичности правок для интернет-магазин «Викишоп». Разработана модель для классификации комментариев на позитивные и негативные с целью выявлять токсичные комментарии и отправлять их на модерацию.

Данные для обучения моделей обработаны:
- проведена лемматизация текста и работа с регулярными выражениями;
- выполнена оценка важности слов величиной TF-IDF. 

Обучены несколько моделей с различными гиперпараметрами: LogisticRegression, CatBoostClassifier. 
Метрика качества выбрана F1. 

Наилучший результат показала модель LogisticRegression c подбором порога: Threshold=0.620, F1-Score=0.85766.
На тестовых данных модель показала следующие результаты: лучшая метрика F1-Score=0.78215 для порога, равного 0.675.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны